# Readout Example

This example notebook shows how a user can read out the data from a board.

This example applies to all board models.

### Naludaq Version  
*Min Version*: `0.21.0`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Compatible Boards
+ `AARDVARCv3`
+ `AARDVARCv4`
+ `AODSoC_AODS`
+ `AODSoC_ASoC`
+ `AODSv2`
+ `ASoCv3`
+ `HDSoCv1_evalr2`
+ `TRBHMv1`
+ `UDC16`
+ `UPAC96`


## Imports and Variables

In [ ]:
# Useful for debugging, but not normally necessary
%load_ext autoreload
%autoreload 2

In [ ]:
import time

import numpy as np

# Imports for board creation and identification
from naludaq.board import Board, startup_board
from naludaq.tools.ftdi import list_ftdi_devices

# Controllers controls one aspect of the board
from naludaq.controllers import get_board_controller

# Imports for data acquisition.
from naludaq.tools.data_collector import get_data_collector

# Imports for helpers (requires installation of the naluexamples package)
from naluexamples.helpers.plotting import set_plot_style, simple_event_plot

The logger setup below will show additional information from the NaluDAQ package. This is useful for debugging purposes, but
may be skipped if not needed.

In [ ]:
import logging


def setup_logger(level=logging.INFO):
    """Setup a basic logger."""
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    handler.setFormatter(
        logging.Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(level)
    suppress = [
        "naludaq.UART",
        "naludaq.FTDI",
    ]
    for name in suppress:
        logging.getLogger(name).setLevel(logging.CRITICAL)
    return logger


try:
    logger.debug("logger already setup")
except:
    logger = setup_logger()

In [ ]:
set_plot_style()

## Create board object

Use the function below to find the serial number of the board you want to connect to.

In [ ]:
list_ftdi_devices()

Modify the settings below to match your board:

In [ ]:
MODEL = "aardvarcv3"
SERIAL_NUMBER = "A904CVKK"
BAUD_RATE = None  # set to None to use max speed

In [ ]:
BOARD = Board(MODEL.lower())

In [ ]:
BAUD_RATE = BAUD_RATE or max(BOARD.params["possible_bauds"].keys())
BOARD.get_ftdi_connection(serial_number=SERIAL_NUMBER, baud=BAUD_RATE)

In [ ]:
startup_board(BOARD)

## Using the Data Collector

The `DataCollector` class is a NaluDAQ tool which is capable of reliably collecting data. It provides significant flexibility through its `iter()` function, which allows for the creation of a data pipeline.

This pipeline can be a bit confusing at first, but is very powerful once understood.

Let's take a look at some simple examples first. Initially we need to set up a `DataCollector` using the `Board` object.

In [ ]:
COLLECTOR = get_data_collector(BOARD)
COLLECTOR.channels = range(BOARD.channels)
COLLECTOR.set_window(windows=10, lookback=10, write_after_trig=10)

In the cell below, we use the `iter()` function to create a special iterator through which we can chain together a pipeline.

The `take(5)` portion specifies that we desire only 5 events from the pipeline, after which the iterator terminates.

The `collect()` function executes the data collection pipeline and collects the 5 events into a list.

Lastly, we plot the 9th (last) event in the result.

In [ ]:
DATA = COLLECTOR.iter(count=10).take(5).collect()
simple_event_plot(DATA[4])

The iterator returned by the `iter()` function provides us with many options in how we wish to piece together our data pipeline. Each stage of the pipeline is executed sequentially on each element of the iterator on the fly; as soon as an event is captured it is passed through the entire pipeline before the next event is captured.

The (non-exhaustive) list of functions available to us are:
- `take(n)` - take only the first `n` elements of the iterator, as in the cell above. All other elements are discarded.
- `map(f)` - apply the function `f` to each element of the iterator, passing the result on through the pipeline. The result of the function does not need to be a list
- `filter(f)` - apply the function `f` to each element of the iterator, passing the result on through the pipeline only if `f` returns `True`.
- `skip(n)` - skip the first `n` elements of the iterator
- `skip_while(n)` - skip elements of the iterator while the function `f` returns `True`
- `take_while(n)` - take elements of the iterator while the function `f` returns `True`
- `enumerate()` - pass the element of the iterator along with its index through the pipeline
- `unenumerate()` - pass only the element of the iterator through the pipeline, discarding the index. An `enumerate()` stage must precede this stage.

Note that the events are passed through the pipeline as quickly as they arrive (disclaimer: up to 10 ms delay), but the stages added by the user can introduce some delay which causes events to be received later than expected, depending on how long the pipeline takes to execute. If the pipeline is slow, it is advisable to either:

1. capture all events first then process the data, or
2. use the `map(f)` function to pass the data to a processing thread.

That being said, let's look at a fancier example. Put together, the cell below will filter out odd-numbered events, take the first 10 events, then discard the first 2.

In [ ]:
DATA = (
    COLLECTOR.iter(count=100)
    .enumerate()                      # 1. add an index to each event so each element is of the form (index, event)
    .filter(lambda x: x[0] % 2 == 1)  # 2. only collect odd events (skip every other event)
    .take_while(lambda x: x[0] < 10)  # 3. only collect the first 10 events (same as take(10))
    .skip(2)                          # 4. skip the first 2 events
    .collect()                        # 5. execute the data collector and collect the events into a list
)
print("Got event numbers: ", [x[0] for x in DATA])

The iterator returned by the `DataCollector` acts as a regular old Python iterator, so it can also be used in a `for` loop. Note that the body of the for loop will be executed in real-time, so you can process the data as it is being captured.

In [ ]:
for i, event in COLLECTOR.iter(count=5).enumerate():
    print(f"Just captured event {i}")

    # some boards are really fast, so let's wait a bit to demonstrate the real-time execution.
    time.sleep(1)

Exit conditions can also be specified in the body of a loop. The example below also demonstrates that built-in functions such as `enumerate` can be used to wrap the `DataCollector` iterator.

In [ ]:
another_iterable = range(100)
for i, event in zip(another_iterable, COLLECTOR.iter(count=10)):
    if i >= 5:
        break
    print(f"Just captured event {i}")

Remember, the `DataCollector` iterator pipeline need not only pass along events, it can pass along anything you wish!

The example below takes the average of each channel and prints the index and the mean value for each event.

In [ ]:
def do_some_processing(event: dict) -> list[float]:
    """Perform some processing on the event and return a list.
    
    In this example we just return the mean over all samples of each channel.
    """
    return [np.mean(channel_data) for channel_data in event["data"]]


_ = (
    COLLECTOR
        .iter(count=10)
        .map(do_some_processing)
        .enumerate()
        .map(print)
        .collect()
)

The `DataCollector` also provides an `iter_inf()` function which returns an infinite iterator over the data. It will continue to return data until a user-defined exit condition is met. This is especially useful when the exit condition is non-deterministic.

In [ ]:
_ = (
    COLLECTOR
        .iter_inf()
        .enumerate()
        .take_while(lambda x: x[0] < 10)
        .collect()
)

## If the DataCollector Suddenly Times Out

Sometimes a board may lock up, or fail to return events for some reason. The `iter()` and `iter_inf()` functions take an optional `attempts` parameter which can be used to adjust how many times the `DataCollector` will attempt to read an event before giving up and raising a `TimeoutError`. The default is `3` attempts, but you can raise this number if you find that your data collection is timing out too often.

Most of the time, getting a `TimeoutError` just means that `attempts` is too low. However, here are some other less-common possibilities:

- you are using older/incompatible versions of naluconfigs/naludaq/firmware
- a background thread is hogging the serial port (mainly a problem when using an older DAQ class)
- the firmware has gotten locked up, in which case running the cell below should fix it. If not, try a power cycle.

In [ ]:
get_board_controller(BOARD).reset_board()